# Step 2: Model Building
Using the sensor data sets explored and constructed in the `1_data_ingestion_and_preparation.ipynb` Jupyter notebook, this notebook loads the data from the Azure Blob container and builds a LSTM network for scenerio described at [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) to predict remaining useful life of aircraft engines. We then store the model for deployment in an Azure web service. We will prepare and build the web service in the `Code/3_operationalization.ipynb` Jupyter notebook.

In [1]:
import keras

# import the libraries
import h5py
import os
import pandas as pd
import numpy as np

import urllib
import glob
import pickle
import re

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0

from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from sklearn import datasets
from keras.layers import Dense, Dropout, LSTM, Activation

# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger
run_logger = get_azureml_logger()
run_logger.log('amlrealworld.predictivemaintenanceforpm.modelbuildingandevaluation','true')

Using TensorFlow backend.


## Load feature data set

We have previously created the labeled data set in the `Code\1_Data Ingestion and Preparation.ipynb` Jupyter notebook. Since the Azure Blob storage account name and account key are not passed between notebooks, you'll need your credentials here again.

In [2]:
# We will store each of these data sets in a local persistance folder
SHARE_ROOT = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']

# These file names detail the data files. 
TRAIN_DATA = 'PM_train_files.pkl'
TEST_DATA = 'PM_test_files.pkl'

# We'll serialize the model in json format
LSTM_MODEL = 'modellstm.json'

# and store the weights in h5
MODEL_WEIGHTS = 'modellstm.h5'


Load the data and dump a short summary of the resulting DataFrame.

In [3]:
train_df = pd.read_pickle(SHARE_ROOT + TRAIN_DATA)
train_df.head(10)

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,RUL,label1,label2,cycle_norm
0,1,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,0,0,0.000000
1,1,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,0,0,0.002770
2,1,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,0,0,0.005540
3,1,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,0,0,0.008310
4,1,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,0,0,0.011080
5,1,6,0.252874,0.416667,0.0,0.0,0.268072,0.292784,0.272113,0.0,...,0.0,0.250000,0.0,0.0,0.651163,0.652720,186,0,0,0.013850
6,1,7,0.557471,0.583333,0.0,0.0,0.382530,0.463920,0.261985,0.0,...,0.0,0.333333,0.0,0.0,0.744186,0.667219,185,0,0,0.016620
7,1,8,0.304598,0.750000,0.0,0.0,0.406627,0.259865,0.316003,0.0,...,0.0,0.250000,0.0,0.0,0.643411,0.574979,184,0,0,0.019391
8,1,9,0.545977,0.583333,0.0,0.0,0.274096,0.434707,0.211850,0.0,...,0.0,0.333333,0.0,0.0,0.705426,0.707539,183,0,0,0.022161
9,1,10,0.310345,0.583333,0.0,0.0,0.150602,0.440375,0.307394,0.0,...,0.0,0.416667,0.0,0.0,0.627907,0.794256,182,0,0,0.024931


In [4]:
test_df = pd.read_pickle(SHARE_ROOT + TEST_DATA)

test_df.head(10)

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.000000,142,0,0
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.002770,141,0,0
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.005540,140,0,0
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.008310,139,0,0
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.011080,138,0,0
5,1,6,0.568966,0.750000,0.0,0.0,0.271084,0.176150,0.217421,0.0,...,0.0,0.333333,0.0,0.0,0.596899,0.624827,0.013850,137,0,0
6,1,7,0.500000,0.666667,0.0,0.0,0.271084,0.268149,0.381330,0.0,...,0.0,0.250000,0.0,0.0,0.550388,0.691798,0.016620,136,0,0
7,1,8,0.534483,0.500000,0.0,0.0,0.400602,0.214737,0.314652,0.0,...,0.0,0.416667,0.0,0.0,0.705426,0.591273,0.019391,135,0,0
8,1,9,0.293103,0.500000,0.0,0.0,0.201807,0.485066,0.506921,0.0,...,0.0,0.250000,0.0,0.0,0.744186,0.770367,0.022161,134,0,0
9,1,10,0.356322,0.416667,0.0,0.0,0.259036,0.309789,0.276671,0.0,...,0.0,0.250000,0.0,0.0,0.565891,0.673571,0.024931,133,0,0


## Modelling

The traditional predictive maintenance machine learning models are based on feature engineering which is manual construction of right features using domain expertise and similar methods. This usually makes these models hard to reuse since feature engineering is specific to the problem scenario and the available data which varies from one business to the other. Perhaps the most attractive part of applying deep learning in the predictive maintenance domain is the fact that these networks can automatically extract the right features from the data, eliminating the need for manual feature engineering.

When using LSTMs in the time-series domain, one important parameter to pick is the sequence length which is the window for LSTMs to look back. This may be viewed as similar to picking window_size = 5 cycles for calculating the rolling features in the [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) which are rolling mean and rolling standard deviation for 21 sensor values. The idea of using LSTMs is to let the model extract abstract features out of the sequence of sensor values in the window rather than engineering those manually. The expectation is that if there is a pattern in these sensor values within the window prior to failure, the pattern should be encoded by the LSTM.

One critical advantage of LSTMs is their ability to remember from long-term sequences (window sizes) which is hard to achieve by traditional feature engineering. For example, computing rolling averages over a window size of 50 cycles may lead to loss of information due to smoothing and abstracting of values over such a long period, istead, using all 50 values as input may provide better results. While feature engineering over large window sizes may not make sense, LSTMs are able to use larger window sizes and use all the information in the window as input. Below, we illustrate the approach.

In [5]:
# pick a large window size of 50 cycles
sequence_length = 50

[Keras LSTM](https://keras.io/layers/recurrent/) layers expect an input in the shape of a numpy array of 3 dimensions (samples, time steps, features) where samples is the number of training sequences, time steps is the look back window or sequence length and features is the number of features of each sequence at each time step. 

In [6]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

In [7]:
# pick the feature columns 
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
key_cols = ['id', 'cycle']
label_cols = ['label1', 'label2', 'RUL']

input_features = test_df.columns.values.tolist()
sensor_cols = [x for x in input_features if x not in set(key_cols)]
sensor_cols = [x for x in sensor_cols if x not in set(label_cols)]
sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

# The time is sequenced along
# This may be a silly way to get these column names, but it's relatively clear
sequence_cols.extend(sensor_cols)

print(sequence_cols)

['setting1', 'setting2', 'setting3', 'cycle_norm', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']


In [8]:
# generator for the sequences
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

In [9]:
# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

(15631, 50, 25)

In [10]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

In [11]:
# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['label1']) 
             for id in train_df['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

(15631, 1)

## LSTM Network

Next, we build a deep network. The first layer is a LSTM layer with 100 units followed by another LSTM layer with 50 units. Dropout is also applied after each LSTM layer to control overfitting. Final layer is a Dense output layer with single unit and sigmoid activation since this is a binary classification problem.

In [12]:
# build the network
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()

model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=nb_out, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 100)           50400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 80,651
Trainable params: 80,651
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
%%time
# fit the network
model.fit(seq_array, label_array, epochs=10, batch_size=200, validation_split=0.05, verbose=1,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')])

Train on 14849 samples, validate on 782 samples
Epoch 1/10
14849/14849 [==============================] - 18s 1ms/step - loss: 0.2631 - acc: 0.8910 - val_loss: 0.1339 - val_acc: 0.9437
Epoch 2/10
14849/14849 [==============================] - 17s 1ms/step - loss: 0.1066 - acc: 0.9553 - val_loss: 0.1357 - val_acc: 0.9425
CPU times: user 2min 3s, sys: 32.4 s, total: 2min 36s
Wall time: 36.5 s


In [15]:
# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Training Accurracy: {}'.format(scores[1]))

15631/15631 [==============================] - 5s 346us/step
Training Accurracy: 0.9431258380073678


In [16]:
# make predictions and compute confusion matrix
y_pred = model.predict_classes(seq_array,verbose=1, batch_size=200)
y_true = label_array
print('Training Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true, y_pred)
cm

15631/15631 [==============================] - 6s 359us/step
Training Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[12517,    14],
       [  875,  2225]])

In [17]:
# compute precision and recall
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print( 'Training precision \t= ', precision, '\nTraining recall \t= ', recall)

Training precision 	=  0.9937472085752568 
Training recall 	=  0.717741935483871


## Model testing
Next, we look at the performance on the test data. Only the last cycle data for each engine id in the test data is kept for testing purposes. In order to compare the results to the template, we pick the last sequence for each id in the test data.

In [18]:
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
seq_array_test_last.shape

(93, 50, 25)

In [19]:
y_mask = [len(test_df[test_df['id']==id]) >= sequence_length for id in test_df['id'].unique()]

In [20]:
label_array_test_last = test_df.groupby('id')['label1'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)
label_array_test_last.shape

(93, 1)

In [21]:
print(seq_array_test_last.shape)
print(label_array_test_last.shape)

(93, 50, 25)
(93, 1)


In [22]:
# test metrics
scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose=2)
print('Accurracy: {}'.format(scores_test[1]))

Accurracy: 0.8817204313893472


In [23]:
# make predictions and compute confusion matrix
y_pred_test = model.predict_classes(seq_array_test_last)
y_true_test = label_array_test_last
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true_test, y_pred_test)
cm

Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[68,  0],
       [11, 14]])

In [24]:
# compute precision and recall
precision_test = precision_score(y_true_test, y_pred_test)
recall_test = recall_score(y_true_test, y_pred_test)
f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
print( 'Precision: ', precision_test, '\n', 'Recall: ', recall_test,'\n', 'F1-score:', f1_test )

Precision:  1.0 
 Recall:  0.56 
 F1-score: 0.717948717948718


## Saving/loading the model  

### Save and test the loaded model locally first

In [25]:
# Save the model for operationalization: https://machinelearningmastery.com/save-load-keras-deep-learning-models/
import os
import h5py
from sklearn import datasets 
 
# save model
# serialize model to JSON
model_json = model.to_json()
with open(LSTM_MODEL, "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(MODEL_WEIGHTS)
print("Model saved")

Model saved


In [26]:
from keras.models import model_from_json

# load json and create model
json_file = open(LSTM_MODEL, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(MODEL_WEIGHTS)
print("Model loaded")

Model loaded


In [27]:
score = loaded_model.predict_proba(seq_array,verbose=1)
print(score.shape)
print(score)

15631/15631 [==============================] - 11s 689us/step
(15631, 1)
[[7.6512061e-04]
 [7.7340921e-04]
 [7.8899635e-04]
 ...
 [9.8971039e-01]
 [9.9007738e-01]
 [9.9042648e-01]]


### Save the files to the shared folder for operationalization
https://docs.microsoft.com/en-us/azure/machine-learning/preview/how-to-read-write-files

#### Ubuntu Linux
/home/<username>/ then navigate to .azureml/share/<exp_acct_name>/<workspace_name>/<proj_name>/

In [28]:
with open(SHARE_ROOT + LSTM_MODEL, 'wt') as json_file:
    json_file.write(model_json)
    print("json file written shared folder")
    json_file.close()

json file written shared folder


In [29]:
model.save_weights(os.path.join(SHARE_ROOT, MODEL_WEIGHTS))

## Conclusion
In the next notebook `Code/3_operationalization.ipynb` Jupyter notebook we will create the functions needed to operationalize and deploy any model to get realtime predictions.